In [1]:
import pandas as pd

# NFL Big Data Bowl Clustering Extra Point

## Import Data

In [2]:
plays = pd.read_csv("../../NFL Data/Data/plays.csv")
games = pd.read_csv("../../NFL Data/Data/games.csv")
players = pd.read_csv("../../NFL Data/Data/players.csv")
track_ep2018 = pd.read_csv("../../NFL Data/Data/tracking_ep18.csv")
track_ep2019 = pd.read_csv("../../NFL Data/Data/tracking_ep19.csv")
track_ep2020 = pd.read_csv("../../NFL Data/Data/tracking_ep20.csv")

In [3]:
tracking_fep = pd.read_csv("../../NFL Data/Data/tracking_fep.csv")

## Preprocessing Players Data

In [4]:
from Preprocessing_makers import preprocess_players

In [5]:
players_df = preprocess_players(players)

## Preprocessing Tracking Data

In [6]:
from Preprocessing_makers import preprocess_tracking, preprocess_football_track


In [7]:
ep_tracking_2018, ep_tracking_2019, ep_tracking_2020 = preprocess_tracking(track_ep2018, track_ep2019, track_ep2020, plays, 'Extra Point')


In [8]:
ep_tracking_ball = preprocess_football_track(ep_tracking_2018, ep_tracking_2019, ep_tracking_2020)


## Preprocessing Play Data

In [9]:
from Preprocessing_makers import preprocess_play
from Cleaning_data import drop_by_index_difference

In [10]:
play_df = preprocess_play(plays)

In [11]:
play_df.value_counts('specialTeamsResult')

specialTeamsResult
Kick Attempt Good           5470
Return                      5207
Touchback                   5156
Fair Catch                  1645
Downed                       834
Out of Bounds                651
Kick Attempt No Good         585
Muffed                       214
Non-Special Teams Result     101
Blocked Kick Attempt          61
Blocked Punt                  39
Kickoff Team Recovery         16
dtype: int64

In [12]:
play_df_1 = drop_by_index_difference(play_df, ep_tracking_ball, 'extra_point_attempt')

In [13]:
play_df_1.value_counts('specialTeamsResult')

specialTeamsResult
Kick Attempt Good       3030
Kick Attempt No Good     188
Blocked Kick Attempt      15
dtype: int64

In [14]:
play_ep_blocked = play_df_1.loc[play_df_1['specialTeamsResult']=='Blocked Kick Attempt']

In [15]:
play_ep_blocked

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,specialTeamsPlayType,specialTeamsResult,kickerId,...,penaltyJerseyNumbers,penaltyYards,preSnapHomeScore,preSnapVisitorScore,passResult,kickLength,kickReturnYardage,playResult,absoluteYardlineNumber,gameClockSeconds
702,2018091611,1690,"M.Nugent extra point is Blocked (S.Barrett), C...",2,0,0,OAK,Extra Point,Blocked Kick Attempt,29874.0,...,NaN,0.0,0,12,NaN,NaN,NaN,0,25,935
5038,2018120210,2278,"H.Butker extra point is Blocked (A.Key), Cente...",2,0,0,KC,Extra Point,Blocked Kick Attempt,45046.0,...,NaN,0.0,7,19,NaN,NaN,NaN,0,25,907
7349,2019091503,2681,"D.Bailey extra point is Blocked (T.Brown), Cen...",3,0,0,MIN,Extra Point,Blocked Kick Attempt,38134.0,...,NaN,0.0,21,16,NaN,NaN,NaN,0,40,2352
7879,2019092209,809,"M.Gay extra point is Blocked (D.Lawrence II), ...",1,0,0,TB,Extra Point,Blocked Kick Attempt,47928.0,...,NaN,0.0,12,3,NaN,NaN,NaN,0,95,21
9557,2019102701,2674,"S.Hauschka extra point is Blocked (D.Barnett),...",3,0,0,BUF,Extra Point,Blocked Kick Attempt,33469.0,...,NaN,0.0,13,17,NaN,NaN,NaN,0,25,2376
10059,2019110306,1629,A.Vinatieri extra point is Blocked (C.Heyward)...,2,0,0,IND,Extra Point,Blocked Kick Attempt,21213.0,...,NaN,0.0,10,16,NaN,NaN,NaN,0,95,942
14140,2020092012,1341,"H.Butker extra point is Blocked (J.Tillery), C...",2,0,0,KC,Extra Point,Blocked Kick Attempt,45046.0,...,NaN,0.0,7,6,NaN,NaN,NaN,0,85,1431
14478,2020092709,1600,"J.Slye extra point is Blocked (I.Rochell), Cen...",2,0,0,CAR,Extra Point,Blocked Kick Attempt,48337.0,...,NaN,0.0,7,15,NaN,NaN,NaN,0,95,1078
14533,2020092711,1788,"R.Succop extra point is Blocked (J.Jones), Cen...",2,0,0,TB,Extra Point,Blocked Kick Attempt,34707.0,...,NaN,0.0,3,23,NaN,NaN,NaN,0,25,1039
14561,2020092712,2011,"G.Zuerlein extra point is Blocked (T.Flowers),...",2,0,0,DAL,Extra Point,Blocked Kick Attempt,38701.0,...,NaN,0.0,16,15,NaN,NaN,NaN,0,25,1265


## Make Extra Point Data Frame

In [16]:
from Preprocessing_makers import make_extraPoint

In [17]:
extra_point_df = make_extraPoint(play_df_1, players_df)

## Freature Engineering

In [31]:
from Feature_engineering import endzone_y_pos, expected_endzone_y_pos, kicker_core_dist, off_center, exp_err_y

### Endzone y-Position

In [19]:
extra_point_df = endzone_y_pos(extra_point_df, ep_tracking_ball)

In [20]:
extra_point_kick_df = extra_point_df[extra_point_df['specialTeamsResult'].isin(['Kick Attempt Good', 'Kick Attempt No Good'])]

In [21]:
extra_point_kick_df.value_counts('specialTeamsResult')

specialTeamsResult
Kick Attempt Good       3030
Kick Attempt No Good     188
dtype: int64

### Expected Trajectory Position

In [22]:
extra_point_df = expected_endzone_y_pos(extra_point_df, ep_tracking_ball, 'extra_point_attempt')



In [23]:
extra_point_df['expected_endzone_y'].isna().sum()

0

### Kicker Core Distance 3

In [24]:
extra_point_df = kicker_core_dist(extra_point_df, ep_tracking_2018, ep_tracking_2019, ep_tracking_2020, ep_tracking_ball, 'extra_point_attempt', k=3)


/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

...

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

In [25]:
extra_point_df['kicker_core_dist_3'].isna().sum()

8

### Kicker Core Distance 1

In [28]:
extra_point_df = kicker_core_dist(extra_point_df, ep_tracking_2018, ep_tracking_2019, ep_tracking_2020, ep_tracking_ball, 'extra_point_attempt', k=1)


/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

...

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

In [29]:
extra_point_df['kicker_core_dist_1'].isna().sum()

8

## 'endzone_y_error'

In [32]:
extra_point_df = exp_err_y(extra_point_df)

## 'endzone_y_off_center'

In [33]:
extra_point_df = off_center(extra_point_df)

### Final Preprocessing Steps

In [26]:
from Preprocessing_makers import preprocess_ep

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [ ]:
def preprocess_ep(ep_plays):
    '''
    This function the ExtraPoint dataframe for clustering.

    Parameters:
    -----------
    ep_plays - ExtraPoint dataframe
    ...

    Returns:
    -----------
    ep_scale - processed ExtraPoint dataframe

    '''
    #reduce number of columns to those with numeric values or one-hot-encode the categoricals
    useful_cols = ['specialTeamsResult', 'yardlineNumber', 'gameClockSeconds', 
                   'penaltyCodes', 'penaltyYards', 'preSnapHomeScore', 
                   'preSnapVisitorScore', 'kicker_height', 'kicker_weight', #'expected_endzone_y', 
                   'endzone_y', 'endzone_y_error','endzone_y_off_center']
    
    columns = ep_plays.columns
    
    useful_cols.extend(col for col in columns if 'kicker_core_dist' in col)
    
    #useful_cols with blockers
    #useful_cols = ['specialTeamsResult', 'yardlineNumber', 'gameClockSeconds', 
                 #   'penaltyCodes', 'penaltyYards', 'preSnapHomeScore', 'preSnapVisitorScore', 
                # 'kicker_height', 'kicker_weight', 'blocker_height', 'blocker_weight',
                # 'expected_endzone_y', 'endzone_y', 'kicker_core_dist']
                
    #need to drop nulls for clustering
    ep_df = ep_plays[useful_cols].dropna()
    #one-hot-encode SpecialTeamsResult and penaltyCodes
    le_str = LabelEncoder()
    le_pc = LabelEncoder()
    ohe_str = le_str.fit_transform(ep_df['specialTeamsResult'])
    ohe_pc = le_pc.fit_transform(ep_df['penaltyCodes'])
    new_eps = ep_df.drop(['specialTeamsResult', 'penaltyCodes'], axis=1)
    #new_eps['specialTeamsResult'] = ohe_str
    #new_eps['penaltyCodes'] = ohe_pc
    
    #scale data, but only non-categorical columns
    scale = StandardScaler()
    ep_scale = scale.fit_transform(new_eps)
    #TO-DO QUESTION: do we want to scale categoricals too? No
    
    #make this back into a data frame
    ep_scale = pd.DataFrame(ep_scale, columns = new_eps.columns)
    
    #add categorical columns back
    ep_scale['specialTeamsResult'] = ohe_str
    ep_scale['penaltyCodes'] = ohe_pc
    #we are running a distance dependent algorithm
    return ep_scale
    

In [34]:
ep_scale = preprocess_ep(extra_point_df)

In [35]:
ep_scale.isna().sum()

yardlineNumber          0
gameClockSeconds        0
penaltyYards            0
preSnapHomeScore        0
preSnapVisitorScore     0
kicker_height           0
kicker_weight           0
endzone_y               0
endzone_y_error         0
endzone_y_off_center    0
kicker_core_dist_3      0
kicker_core_dist_1      0
specialTeamsResult      0
penaltyCodes            0
dtype: int64

## Clustering

In [36]:
import hdbscan

import matplotlib.pyplot as plt
import seaborn as sns

In [37]:
model = hdbscan.HDBSCAN()

In [38]:
model_results = model.fit(ep_scale)

In [39]:
model_results.labels_.max()

3

In [40]:
ep_scale['cluster_id'] = model_results.labels_

In [41]:
ep_scale['cluster_id'].value_counts()

 3    3102
 1      26
-1      25
 2      25
 0      25
Name: cluster_id, dtype: int64

In [ ]:
plt.figure(figsize = (16,18))
model.condensed_tree_.plot(select_clusters=True, selection_palette=sns.color_palette())
plt.show()

In [42]:
cluster0 = ep_scale[ep_scale['cluster_id']==0]
cluster1 = ep_scale[ep_scale['cluster_id']==1]
cluster2 = ep_scale[ep_scale['cluster_id']==2]
cluster3 = ep_scale[ep_scale['cluster_id']==3]

outliers = ep_scale[ep_scale['cluster_id']==-1]

In [43]:
cluster0['specialTeamsResult'].value_counts()

1    25
Name: specialTeamsResult, dtype: int64

In [44]:
cluster1['specialTeamsResult'].value_counts()

1    26
Name: specialTeamsResult, dtype: int64

In [45]:
cluster2['specialTeamsResult'].value_counts()

1    23
2     2
Name: specialTeamsResult, dtype: int64

In [46]:
cluster3['specialTeamsResult'].value_counts()

1    2920
2     179
0       3
Name: specialTeamsResult, dtype: int64

In [47]:
outliers['specialTeamsResult'].value_counts()

1    15
2     7
0     3
Name: specialTeamsResult, dtype: int64